In [ ]:
from analysis_2019_08 import RCTRun
from circular_buffer import CircularBuffer
from precisionVisualizer import Ping, PingFactory
import numpy as np
import pyfftw
import json
import os
import scipy as sp
import struct
import matplotlib.pyplot as plt; plt.ion()

In [2]:
dataPath = '/media/ntlhui/FA56-CFCD/2019.08.09.Successful_Night_Tracking/RUN_000069/'
run = RCTRun(dataPath)

In [3]:
dataFiles = sorted(run.getDataFiles())
assert(len(dataFiles) > 0)
assert(run.getCenterFreq() is not None)
assert(len(run.getFreqs()) > 0)
assert(run.getSamplingFreq() is not None)
assert(run.getPingWidth() is not None)
assert(run.getSamplingFreq() is not None)
assert(run.getMinSNR() is not None)
assert(run.getStartTime_ms() is not None)
assert(run.getPingFactor()[0] is not None)
assert(run.getPingFactor()[1] is not None)

FFT_LEN = 2048

fft_bins = ((np.array(run.getFreqs()) - run.getCenterFreq()) / (run.getSamplingFreq() / 2) * FFT_LEN).astype(np.int)
print(run.getFreqs())
print(fft_bins)

# frequencyStep is the number of SDR samples between FFT windows
frequencyStep = 256
# fftFrequency is the sample frequency of FFTs in Hz
fftFrequency = run.getSamplingFreq() / frequencyStep
print("FFT Frequency: %f Hz" % (fftFrequency))
# integrateStep is the number of FFT samples between integration windows
integrateStep = 5
# integrateFactor is number of FFT samples to integrate
integrateFactor = 5
# integrateFactor = int(6e-3 * processable_runs[1].getSamplingFreq() / FFT_LEN * frequencyStepFactor)
# classifierInputFrequency is the sample frequency of the input signal to the classifier stage
classifierInputFreq = fftFrequency / integrateStep
print("Classifier Input Frequency: %f Hz" % (classifierInputFreq))
# pingWidth_samp is the floor of the ping width in samples at the classifier sample frequency
pingWidth_samp = int(run.getPingWidth() / 1000. * classifierInputFreq)
print("Ping Width: %d samples" % (pingWidth_samp))
dataLen = pingWidth_samp * 2
print("Databuf Length: %d samples" % (dataLen))
maximizerLen = int(0.1 * classifierInputFreq)
medianLen = int(1 * classifierInputFreq)
idLen = int(0.5 * classifierInputFreq)
readLen = int(frequencyStep)

integrateCounter = 0
idIdx = 0
sampIdx = 0
integrator = CircularBuffer(integrateFactor)
dataBuf = CircularBuffer(dataLen)
peakHistory = CircularBuffer(maximizerLen)
peaks = CircularBuffer(medianLen)
idSignal = CircularBuffer(idLen)
dataFile = open(dataFiles[0], 'rb')
minSNRVec = np.ones(fft_bins.shape) * run.getMinSNR()
fftBuffer = CircularBuffer(FFT_LEN)
for i in range(FFT_LEN):
    fftBuffer.add(np.array([0+0j]))
pings = []
outputSignal = []
thresholdSignal = []
integratorSignal = []

pyfftw.config.NUM_THREADS = 7

pyfftw.config.PLANNER_EFFORT = 'FFTW_MEASURE'
fft_in = pyfftw.empty_aligned(FFT_LEN, dtype='complex64')
fft_out = pyfftw.empty_aligned(FFT_LEN, dtype='complex64')

fft_object = pyfftw.FFTW(fft_in, fft_out, threads=7)

print("Rejecting pings narrower than %f or wider than %f" % (run.getPingFactor()[0] * run.getPingWidth(), run.getPingFactor()[1] * run.getPingWidth()))
print("Rejecting pings with amplitude %f less than noise" % (run.getMinSNR()))

print("FFT Frequency: %.3f Hz" % (fftFrequency))
print("Integration length: %d samples" % (integrateFactor))
print("Classifier Frequency: %.3f Hz" % (classifierInputFreq))
print("Ping width: %d samples" % (pingWidth_samp))

[172642000]
[387]
FFT Frequency: 5859.375000 Hz
Classifier Input Frequency: 1171.875000 Hz
Ping Width: 42 samples
Databuf Length: 84 samples
Rejecting pings narrower than 27.000000 or wider than 54.000000
Rejecting pings with amplitude 1.000000 less than noise
FFT Frequency: 5859.375 Hz
Integration length: 5 samples
Classifier Frequency: 1171.875 Hz
Ping width: 42 samples


In [4]:
localizeFile = run.getLocalizeFile()[0]
pFact = PingFactory()
pings = []
with open(localizeFile) as lFile:
    for line in lFile:
        pingCandidate = pFact.fromJSON(line)
        if pingCandidate is not None:
            pings.append(pingCandidate)

In [5]:
dataFiles = sorted(run.getDataFiles())
dataFileLengths = {f:os.path.getsize(f) for f in dataFiles}
mode = sp.stats.mode(np.array(list(dataFileLengths.values())))
assert(len(mode.mode) == 1)
assert(mode.count[0] == len(dataFiles) - 1)
dataFileLen = mode.mode[0]

In [6]:
def timeStampToFile(t_ms):
    origin = run.getStartTime_ms()
    delta_ms = t_ms - origin
    sampleFreq = run.getSamplingFreq()
    delta_sample = delta_ms / 1000. * sampleFreq
    return delta_sample / dataFileLen

In [16]:
filesOfInterest = list(set([int(timeStampToFile(ping.time)) for ping in pings]))
filesOfInterest

[1, 2, 4, 5]

In [28]:
outputFFT = []
with open(dataFiles[filesOfInterest[0]], 'rb') as dataFile:
    while True:
        data = dataFile.read(readLen * run.getSampleWidth() * 2)
        if len(data) < readLen * run.getSampleWidth() * 2:
            break
        block = struct.unpack("hh" * int(readLen), data)
        [fftBuffer.add(np.array([(block[2 * i] + block[2 * i + 1] * 1j) / run.getDataScalar()])) for i in range(int(readLen))] # This is pretty slow!!!
        sampIdx += FFT_LEN
        a = fftBuffer.to_array()
        for i in range(FFT_LEN):
            fft_in[i] = a[i]
        complexFFT = fft_object() / FFT_LEN
        outputFFT.append(complexFFT)

KeyboardInterrupt: 